<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/sdxl_0_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
from IPython.display import clear_output
from pathlib import Path

!npm install -g localtunnel > /dev/null 2>&1
OPTIONS = {}

USE_GOOGLE_DRIVE = False
UPDATE_COMFY_UI = False
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

clear_output()

#2

MODEL_LINK_1 = "https://huggingface.co/modelslover/notsdxl/resolve/main/sd_xl_base_0.9%20(3).safetensors"
MODEL_SAFETENSORS_1 = True
MODEL_LINK_2 = "https://huggingface.co/modelslover/notsdxl/resolve/main/diffusion_pytorch_model.fp16.safetensors"
MODEL_SAFETENSORS_2 = True

MODEL_LINKS = [
    {
        'link': MODEL_LINK_1,
        'safetensors': MODEL_SAFETENSORS_1
    },
    {
        'link': MODEL_LINK_2,
        'safetensors': MODEL_SAFETENSORS_2
    },

]

if all(model['link'] for model in MODEL_LINKS):
  pth = '/content/ComfyUI/models/checkpoints/'
  for i, model in enumerate(MODEL_LINKS):
    link = model['link']
    safetensors = model['safetensors']

    if safetensors:
        modelname = "model.safetensors"
    else:
        modelname = "model.ckpt"

    dwnld = pth + f"model{i+1}.{modelname.split('.')[1]}"
    print(f'[1;32mDownloading model {i+1}...')
    !gdown --fuzzy -O $dwnld "$link"
    print(f'[1;32mModel {i+1} downloaded!')

  print('[1;32mAll models downloaded successfully!')
else:
  print('[1;31mPlease provide links and corresponding safetensors parameters for all models!')


In [ ]:
#@title ##**Open** { display-mode: "form" }

!fuser -k 8188/tcp
import subprocess
import threading
import time
import socket
import urllib.request
from IPython.display import HTML, display

button_displayed = False

def iframe_thread(port):
    global button_displayed
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    if not button_displayed:
        password = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

        p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)
        for line in p.stdout:
            if line.decode().startswith('your url is:'):
                url = line.decode().split(': ')[1].strip()
                break

        # Создаем HTML с кнопками
        html = f"""
        <script>
        function copyPassword() {{
            navigator.clipboard.writeText("{password}");
            alert("Password copied to clipboard!");
        }}
        </script>
        <button onclick="window.open('{url}', '_blank')">Open Tunnel</button>
        <button onclick="copyPassword()">Copy Password</button>
        <p>Password: {password}</p>
        """
        display(HTML(html))
        button_displayed = True

if not button_displayed:
    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server > /dev/null 2>&1